# TP1 Science des donnees - Chakirou ALABANI (alac_3501) - Amadou Selle NDIAYE (ndia2909)

In [2]:
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head()

(801, 20532)


,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [20]:
data_labels = pd.read_csv('labels.csv')
print(data_labels.shape)
data_labels.head()

(801, 2)


,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA


In [38]:
print(data_labels['Class'][0])

PRAD


# Merge

In [36]:
df = pd.merge(data,data_labels, on='Unnamed: 0',how='inner')
df.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,Class
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0,PRAD
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0,LUAD
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0,PRAD
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0,PRAD
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0,BRCA


# Pre-traitement

In [23]:
#Suppression de 'Unnamed: 0'
df.drop(columns=['Unnamed: 0'],axis=1, inplace=True)

In [52]:
#Suppression de toutes les colonnes avec plus de 70 % de valeurs nulles
colonnes_a_supprimer=[]

for i in df.columns:
    s=0
    for j in range(df.shape[0]):
        if df[i][j] == 0:
            s+=1
    if (s/df.shape[0]) >= 0.70:
        colonnes_a_supprimer.append(i)

print(colonnes_a_supprimer)
len(colonnes_a_supprimer)

['gene_0', 'gene_5', 'gene_8', 'gene_9', 'gene_15', 'gene_16', 'gene_17', 'gene_23', 'gene_24', 'gene_37', 'gene_41', 'gene_42', 'gene_43', 'gene_168', 'gene_169', 'gene_222', 'gene_235', 'gene_236', 'gene_237', 'gene_239', 'gene_254', 'gene_255', 'gene_266', 'gene_274', 'gene_283', 'gene_284', 'gene_287', 'gene_288', 'gene_290', 'gene_353', 'gene_356', 'gene_404', 'gene_479', 'gene_497', 'gene_608', 'gene_610', 'gene_616', 'gene_617', 'gene_779', 'gene_787', 'gene_824', 'gene_841', 'gene_842', 'gene_845', 'gene_859', 'gene_883', 'gene_921', 'gene_1047', 'gene_1079', 'gene_1083', 'gene_1084', 'gene_1090', 'gene_1091', 'gene_1141', 'gene_1184', 'gene_1207', 'gene_1217', 'gene_1261', 'gene_1288', 'gene_1295', 'gene_1311', 'gene_1312', 'gene_1370', 'gene_1383', 'gene_1388', 'gene_1389', 'gene_1392', 'gene_1469', 'gene_1480', 'gene_1497', 'gene_1506', 'gene_1524', 'gene_1527', 'gene_1540', 'gene_1541', 'gene_1576', 'gene_1580', 'gene_1581', 'gene_1585', 'gene_1614', 'gene_1624', 'gene_1628

2109

In [53]:
#Representation des classes
df['Class'].value_counts()

Class
BRCA    300
KIRC    146
LUAD    141
PRAD    136
COAD     78
Name: count, dtype: int64

# Calcul des distances 

# Notes

- Je vais merge les deux datasets pour avoir la correspondance de chaque espece a sa classe
- Apres je vais faire un pre-traitement vu qu'il y'a beaucoup de colonnes qui n'ont pas assez d'informations (Beaucoup de 0). La colonne Unamed:0 pourra etre enleve aussi.
- Ensuite je pourrai proceder au calcul de distance